Long Short-Term Memory network is a type of Recurrent Neural Network

In [2]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [3]:
# fix random seed for reproducibility
numpy.random.seed(7)


In [4]:
# load the dataset
dataframe = pandas.read_csv('netreachvolumes.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [5]:
dataframe.describe()

,Net Reach
count,1.710000e+02
mean,9.597096e+06
std,6.485620e+06
min,1.035764e+06
25%,5.870565e+06
50%,8.173524e+06
75%,1.103199e+07
max,4.972979e+07


In [6]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [7]:
# split into train and test sets - train set ends on May 12
train_size = 61
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size-2:len(dataset),:]
print(len(train), len(test))

(61, 112)


In [8]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [9]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [10]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [11]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=10, batch_size=1, verbose=2)

Epoch 1/10
1s - loss: 0.0594
Epoch 2/10
1s - loss: 0.0346
Epoch 3/10
1s - loss: 0.0231
Epoch 4/10
1s - loss: 0.0189
Epoch 5/10
1s - loss: 0.0178
Epoch 6/10
1s - loss: 0.0177
Epoch 7/10
1s - loss: 0.0176
Epoch 8/10
1s - loss: 0.0176
Epoch 9/10
1s - loss: 0.0175
Epoch 10/10
1s - loss: 0.0175


In [12]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 6420830.35 RMSE
Test Score: 6777212.30 RMSE


In [13]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1-2:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
#plt.show()

In [14]:
trainPredictPlot[59]

array([ 14032136.], dtype=float32)

In [15]:
testPredictPlot[59]

array([ nan], dtype=float32)

In [16]:
import numpy as np

In [17]:
np.count_nonzero(~np.isnan(testPredictPlot))

110

In [18]:
testPredictPlot[60]

array([ 14966257.], dtype=float32)

In [19]:
trainPredictPlot[60:] = testPredictPlot[60:]

In [20]:
trainPredictPlot

array([[       nan],
       [ 12249573.],
       [ 12651656.],
       [ 12185246.],
       [ 12483526.],
       [ 13020171.],
       [ 12860371.],
       [ 13791076.],
       [ 13060127.],
       [ 12671042.],
       [ 12705147.],
       [ 12709331.],
       [ 12601954.],
       [ 12860981.],
       [ 12420385.],
       [ 12212492.],
       [ 12299818.],
       [ 12219980.],
       [ 12227456.],
       [ 12235809.],
       [ 12417684.],
       [ 12356925.],
       [ 12176556.],
       [ 12219850.],
       [ 12256937.],
       [ 12478849.],
       [ 12461597.],
       [ 12371209.],
       [ 13267556.],
       [ 13180627.],
       [ 13446882.],
       [ 13016415.],
       [ 13645996.],
       [ 13109773.],
       [ 13068709.],
       [ 12899793.],
       [ 12449484.],
       [ 12488706.],
       [ 13134220.],
       [ 12344841.],
       [ 12942468.],
       [ 14866024.],
       [ 18031936.],
       [ 12518429.],
       [ 12778576.],
       [ 13126175.],
       [ 12908876.],
       [ 1682

In [21]:
dataset = scaler.inverse_transform(dataset)

In [22]:
trainPredictPlot[0] = dataset[0]

In [23]:
line_up, = plt.plot(trainPredictPlot, label='predicted')
line_down, = plt.plot(dataset, label='observed')
plt.legend(labels=[line_up, line_down])
plt.show()

In [24]:
differences = trainPredictPlot -dataset

In [25]:
import numpy
differences = numpy.asarray(differences)

In [26]:
import pandas as pd
df = pd.read_csv('netreachvolumes.csv')
df.columns = [['date','volume']]
df['predicted'] = trainPredictPlot
df['difference'] = differences

In [27]:
df.to_csv("netreachdifferences.csv")